In [1]:
import pandas as pd
import ipywidgets as widgets
import ipydatetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter, MinuteLocator, date2num
from IPython.display import HTML
import datetime

# preamble
wdir = ["N","NNE","NE","ENE","E","ESE","SE","SSE","S","SSW","SW","WSW","W","WNW","NW","NNW","---"]
OB = ["16 Hz","31.5 Hz","63 Hz","125 Hz","250 Hz","500 Hz","1 kHz","2 kHz","4 kHz","8 kHz","16 kHz"]
pos = -1
table1 = 0

%matplotlib widget
fig = plt.figure(figsize=(10,4))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122, sharex=ax1)

def getdata():
    dir = "Data//"
    lt = [[[] for i in range(2)] for j in range(8)]
    weather = pd.read_excel(dir+"Weather.xlsx")
    flis = ["LT1", "LT2", "LT3", "LT4", "LT5", "LT6", "LT7", "LT8"]
    sheet = ["Leq","L90(LN4)"]
    smin = []
    for i, j in enumerate(flis):
        for k,l in enumerate(sheet):
            lt[i][k] = pd.read_excel(dir+flis[i]+".xlsx", sheet_name=l)
            lt[i][k]['Start Time'] = pd.to_datetime(lt[i][k]['Start Time'].astype(str))
            lt[i][k] = pd.merge_asof(lt[i][k],weather,left_on="Start Time", right_on="Time")
            lt[i][k] = lt[i][k].drop(labels=['Time'],axis=1)
            lt[i][k] = lt[i][k].set_index('Start Time')
            smin.append(lt[i][k].index.min())
            lt[i][k] = lt[i][k].tail(lt[i][k].shape[0] - 24) # removing first 6 hours from tail
            lt[i][k] = lt[i][k].head(lt[i][k].shape[0] - 24) # removing first 6 hours from head
    start = min(smin)
    return start, lt

def pr1(n, lw, c, c2, w, tt):
    global start
    global pos
    n = int(n-1)
    if lw == 'Leq':
        lwv = 0
    else:
        lwv = 1
    ltall = lt[n][lwv]
    ltmin = ltall.index.min()
    if pos != n:
        tt = ltmin
        pos = n
    start = datechange(tt)
    end = start+pd.Timedelta(hours=12)
    ltall = ltall[start:end] # between two datetimes
    ltall = ltall[ltall["Dir"].isin(w)]
    if c:
        ltall = ltall[ltall["Rain"]<0.2]
    if c2:
        ltall = ltall[ltall["Speed"]<=5]
    return ltall, start, end, lw

def plot(n, lw, c, c2, w, tt):
    ltall, start, end, lw = pr1(n, lw, c, c2, w, tt)
    ax1.clear(), ax2.clear()
    if lw == 'Leq':
        ax1.set_ylabel('L$_{EQ}$ dBA')
    else:
        ax1.set_ylabel('L$_{90}$ dBA')
    ax1.scatter(ltall.index,ltall['Main'],c='black',marker='.')
    ltall_OB = ltall[OB].T
    start, end = date2num(start), date2num(end)
    ax2.imshow(ltall_OB, aspect="auto", origin='lower', extent=[start,end,0,len(OB)])
    myFmt = DateFormatter('%H:%M')
    minutes = MinuteLocator(interval = 240)
    ax1.xaxis.set_major_locator(minutes)
    ax1.xaxis.set_major_formatter(myFmt)
    tickloc = [x + 0.5 for x in range(0, len(OB))]
    ax2.yaxis.set(ticks=tickloc, ticklabels=OB)
    ax2.set_yticklabels(OB)
    ax1.set_xlabel('Time')
    ax2.set_xlabel('Time')
    fig.autofmt_xdate()
    #fig.show()

def table(n, lw, c, c2, w, tt):
    global table1
    ltall, start, end, lw = pr1(n, lw, c, c2, w, tt)
    ltall_av,ltall_min,ltall_max = ltall['Main'].mean(),ltall['Main'].min(),ltall['Main'].max()
    df = {'Date / Time':[start],'Arithmetic Mean (dBA)':ltall_av,'Min (dBA)':ltall_min,'Max (dBA)':ltall_max}
    table1 = pd.DataFrame(df).round(1)
    table1 = HTML(table1.to_html(index=False,notebook=True))

def datechange(tt):
    global start
    start = tt
    dt.value = tt
    return start

start, lt = getdata()
# layout
box_layout = widgets.Layout(display='flex',flex_flow='column',align_items='stretch')
box_layout2 = widgets.Layout(display='inline-flex',flex_flow='row',align_items='stretch',margin='0px 0px 10px 45px')
# widgets
aw = widgets.Select(options=list(range(1,9)),description='LT: ',disabled=False)
lw = widgets.ToggleButtons(options=['Leq','L90'],description='Stat. level: ',disabled=False, layout=dict(width='250px'))
dt = ipydatetime.NaiveDatetimePicker(description='Choose date / time',disabled=False,value=start)
cw = widgets.Checkbox(value=False,description='Filter preceipition',disabled=False)
cw2 = widgets.Checkbox(value=False,description='Filter wind speeds above 11mph',disabled=False)
wd = widgets.SelectMultiple(options=wdir,value=wdir,description='wind dir', disabled=False)
button = widgets.Button(description="Click Me!")
ui = widgets.VBox([widgets.HBox([button, dt, lw, widgets.VBox([cw,cw2],layout=box_layout)],layout=box_layout2),widgets.HBox([aw, wd])],layout=box_layout)

# interactive output
output = widgets.interactive_output(table, {'n': aw, 'lw': lw, 'c': cw, 'c2': cw,'w':wd,'tt':dt})
output2 = widgets.interactive_output(plot, {'n': aw, 'lw': lw, 'c': cw, 'c2': cw2,'w':wd,'tt':dt})
#output2 = widgets.HBox([tablewidget],layout=widgets.Layout(margin='0px 0px 0px 90px'))

# output
#output = widgets.Output()
display(output2, output, ui)

def update(b):
    with output:
        output.clear_output()
        display(table1)

button.on_click(update)